In [7]:
import sys
import os
from pathlib import Path
import numpy as np

# --- 1. 強制精確定位專案根目錄 ---
def get_project_root():
    current = Path(os.getcwd()).resolve()
    # 往上找，直到找到包含 Resemblyzer 的那個資料夾為止
    for p in [current] + list(current.parents):
        if (p / "Resemblyzer").exists():
            return p
    return current

root_path = get_project_root()
res_path = root_path / "Resemblyzer"

# 加入系統路徑
if str(res_path) not in sys.path:
    sys.path.append(str(res_path))

print(f"📍 目前工作目錄: {os.getcwd()}")
print(f"🎯 定位的專案根目錄: {root_path}")
print(f"資料夾檢查:")
print(f" - Resemblyzer: {'✅ 找到' if res_path.exists() else '❌ 未找到'}")
print(f" - Enrollment: {'✅ 找到' if (root_path / 'enrollment_wavs').exists() else '❌ 未找到'}")

# 測試載入
try:
    from resemblyzer import VoiceEncoder, preprocess_wav
    print("\n✅ Resemblyzer 載入成功！")
except ImportError as e:
    print(f"\n❌ 載入失敗: {e}")


📍 目前工作目錄: c:\Users\bexo6\OneDrive\桌面\azure_speech
🎯 定位的專案根目錄: C:\Users\bexo6\OneDrive\桌面\azure_speech
資料夾檢查:
 - Resemblyzer: ✅ 找到
 - Enrollment: ✅ 找到

✅ Resemblyzer 載入成功！


In [8]:
import librosa

# --- 2. 初始化模型 ---
encoder = VoiceEncoder()

# --- 3. 定義處理 MP4 的函數 ---
def preprocess_mp4(mp4_path):
    # 使用 librosa 讀取影片中的音軌，sr=16000 是 Resemblyzer 的標準採樣率
    wav, _ = librosa.load(str(mp4_path), sr=16000)
    return wav

# --- 4. 處理員工 A 的唸稿音檔 (Enrollment) ---
# 指向外層的 enrollment_wavs 資料夾
enroll_folder = root_path / "enrollment_wavs"
enroll_files = ["clinics_3060.wav", "clinics_3090.wav"]

print("正在提取員工聲紋特徵...")
embs = []
for f in enroll_files:
    f_path = enroll_folder / f
    if f_path.exists():
        # preprocess_wav 處理標準 wav 檔
        wav = preprocess_wav(f_path)
        embs.append(encoder.embed_utterance(wav))
        print(f"  - 已加入: {f}")
    else:
        print(f"  - ⚠️ 找不到檔案: {f_path}")

# 取平均值作為該員工的基準聲紋 (Embedding)
if embs:
    staff_A_emb = np.mean(embs, axis=0)
    print("✅ 員工 A 聲紋特徵庫建立完成。")
else:
    print("❌ 無法建立聲紋庫，請檢查音檔路徑。")

# --- 5. 處理現場對談音檔 (Testing) ---
# MP4 檔案在根目錄
test_mp4_path = root_path / "20250922114652_2025-09-22_11-46-51_唐育晨.mp4"

print(f"\n正在分析現場對談: {test_mp4_path.name}")
if test_mp4_path.exists():
    # 讀取 MP4
    test_wav = preprocess_mp4(test_mp4_path)
    # 提取現場對話的特徵
    live_emb = encoder.embed_utterance(test_wav)

    # --- 6. 比對 ---
    # 計算餘弦相似度 (Cosine Similarity)
    similarity = np.dot(staff_A_emb, live_emb)
    
    print("-" * 30)
    print(f"🔍 聲紋比對得分: {similarity:.4f}")
    
    # 門檻值建議：> 0.75 為高度匹配，0.65 - 0.75 為中度匹配
    if similarity > 0.75:
        print("💡 結論：偵測到強烈的員工 A 聲音特徵。")
    elif similarity > 0.65:
        print("💡 結論：有部分特徵符合，但可能包含較多客人聲音或雜音。")
    else:
        print("💡 結論：匹配度低。")
else:
    print(f"❌ 找不到 MP4 檔案: {test_mp4_path}")

Loaded the voice encoder model on cpu in 0.02 seconds.
正在提取員工聲紋特徵...
  - 已加入: clinics_3060.wav
  - 已加入: clinics_3090.wav
✅ 員工 A 聲紋特徵庫建立完成。

正在分析現場對談: 20250922114652_2025-09-22_11-46-51_唐育晨.mp4


C:\Users\bexo6\AppData\Local\Temp\ipykernel_27872\2051656683.py:9: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, _ = librosa.load(str(mp4_path), sr=16000)
c:\Users\bexo6\OneDrive\桌面\azure_speech\.venv\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


------------------------------
🔍 聲紋比對得分: 0.7254
💡 結論：有部分特徵符合，但可能包含較多客人聲音或雜音。
